<a href="https://colab.research.google.com/github/FrancescoSarandrea/deeplearning/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Iniziamo installando delle linrerie che fanno _scraping_ da Twitter; utilizzeremo la libreria **Transformer**, che ci permette di avere accesso a tuttti i modelli di analisi.

In [ ]:
!pip install snscrape
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.0 MB/s 
     |████████████████████████████████| 101 kB 13.2 MB/s 
     |████████████████████████████████| 6.6 MB 41.9 MB/s 
     |████████████████████████████████| 596 kB 70.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
import re
import transformers 
from transformers import pipeline
import tensorflow as tf

In [ ]:
sentiment=pipeline(model='cardiffnlp/twitter-roberta-base-sentiment')

Downloading:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Scriviamo una funzione per scaricare i tweet

In [ ]:
# la funzione scarica un certo numero di tweet che contengono uno 
# specifico search term e sono nel periodo indicato
def tweets(n_tweets,search_term,start_date,end_date):
  tweets_list2=[]
  for i,tweet in enumerate(sntwitter.TwitterSearchScraper(f'{search_term} since:{start_date} until {end_date}').get_items()):
    if i>n_tweets:
      break
    tweets_list2.append([tweet.date,tweet.id,tweet.content])
  tweet_df=pd.DataFrame(tweets_list2,columns=['Datetime','Tweet_id','Text'])
  return tweet_df

In [ ]:
df_camille=tweets(5000,'camille vasquez','2022-05-20','2022-05-30')
df_camille

,Datetime,Tweet_id,Text


In [ ]:
df=df_camille.assign(Text=lambda x: x['Text'].str.lower())
df=df.loc[df['Text'].str.contains('amber|heard|johnny|depp|shannon|curry')]

In [ ]:
df

,Datetime,Tweet_id,Text


In [ ]:
# rimuoviamo gli URL dai tweet
df=df.assign(Text=lambda x:x['Text'].apply(lambda s: re.sub(r'\w\/{2}[\d\w-]+(\.[\d\w-]+*(?:(\/[^\s/]*))*','',s)))

Tronchiamo il testo a 500 caratteri

In [ ]:
df['Text']=df['Text'].apply(lambda s: s[0:500])

Scriviamo la funzione che estragga il sentiment dai tweet

In [ ]:
df=df.assign(sentim=lambda x: x['Text'].apply(lambda s:sentiment(s)))

In [ ]:
df=df.assign(sentim_label=lambda x: x['sentim'].apply(lambda s:s[0]['label']))
df=df.assign(sentim_label=lambda x: x['sentim'].apply(lambda s:s[0]['score']))

In [ ]:
df.head()

In [ ]:
df['sentiment_label'].value_counts()